In [8]:
import sys
import os

# Add the GARCH directory to Python path
garch_dir = '/Users/hraj/GitHub/RoughVolatility/Rough-Volatility/GARCH'
if garch_dir not in sys.path:
    sys.path.insert(0, garch_dir)
print(f"Added to path: {garch_dir}")

Added to path: /Users/hraj/GitHub/RoughVolatility/Rough-Volatility/GARCH


In [9]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from simulate_data import generate_sample_data
from garch_analysis import GARCHAnalyzer
from visualization import VolatilityVisualizer

### Function to get S&P 500 data from Yahoo Finance 

In [10]:
def get_market_data(symbol: str = "^GSPC", years: int = 4) -> pd.DataFrame:
    """
    Fetch market data from Yahoo Finance and prepare it for GARCH analysis.
    
    Parameters:
    -----------
    symbol : str
        Yahoo Finance ticker symbol (default: ^GSPC for S&P 500)
    years : int
        Number of years of historical data to fetch
        
    Returns:
    --------
    pd.DataFrame
        DataFrame containing returns and prices
    """
    # Calculate start date
    end_date = datetime.now()
    start_date = end_date - timedelta(days=years*365)
    
    # Fetch data
    ticker = yf.Ticker(symbol)
    data = ticker.history(start=start_date, end=end_date)
    
    # Calculate returns and handle NaN values
    data['returns'] = data['Close'].pct_change()
    
    # Remove any NaN values
    data = data.dropna()
    
    # Create DataFrame with required format
    df = pd.DataFrame({
        'returns': data['returns'],
        'prices': data['Close']
    })
    
    print(f"\nMarket Data Summary:")
    print(f"Total observations: {len(df)}")
    print(f"Date range: {df.index[0].strftime('%Y-%m-%d')} to {df.index[-1].strftime('%Y-%m-%d')}")
    print(f"Average daily return: {df['returns'].mean():.6f}")
    print(f"Daily volatility: {df['returns'].std():.6f}")
    
    return df

### Function for performing EDA on the given dataset

In [11]:
def analyze_data(data: pd.DataFrame, title: str) -> GARCHAnalyzer:
    """
    Analyze the given dataset and create visualizations.
    """
    print(f"\nAnalyzing {title}...")
    
    # Create analyzer and fit model
    analyzer = GARCHAnalyzer(data['returns'])
    params = analyzer.fit_model()
    
    # Get conditional volatility
    data['conditional_volatility'] = analyzer.get_conditional_volatility()
    
    # Print model parameters
    print("\nEstimated GARCH parameters:")
    for key, value in params.items():
        print(f"{key}: {value:.4f}")
    
    # Calculate return statistics
    stats = {
        'mean': data['returns'].mean(),
        'std': data['returns'].std(),
        'skewness': data['returns'].skew(),
        'kurtosis': data['returns'].kurtosis(),
        'min': data['returns'].min(),
        'max': data['returns'].max(),
        'acf_squared_returns': pd.Series(data['returns']**2).autocorr()
    }
    
    print("\nReturn series statistics:")
    for key, value in stats.items():
        print(f"{key}: {value:.4f}")
    
    # Create visualizations
    prefix = title.lower().replace(' ', '_').replace('(', '').replace(')', '')
    visualizer = VolatilityVisualizer(data)
    
    print(f"\nPlotting returns and volatility for {title}...")
    visualizer.plot_returns_and_volatility(title_prefix=prefix)
    
    print(f"\nPlotting evidence of volatility clustering for {title}...")
    visualizer.plot_volatility_clustering(title_prefix=prefix)
    
    print(f"\nPlotting returns distribution for {title}...")
    visualizer.plot_returns_distribution(title_prefix=prefix)
    
    if 'conditional_volatility' in data.columns:
        visualizer.plot_qq_plot(title_prefix=prefix)
    
    return analyzer


In [12]:
np.random.seed(42)

## 1. Generating simulated data...

In [13]:
print("\nGenerating GARCH process data (with volatility clustering)...")
garch_params = {
    'omega': 0.05,   # Small constant term
    'alpha': 0.15,   # Impact of past squared returns
    'beta': 0.80     # High persistence in volatility
}

# First analyze without shock
print("\nAnalyzing GARCH process without shock...")
data_garch = generate_sample_data(n_samples=1000, garch_params=garch_params)
analyzer_no_shock = analyze_data(data_garch, "GARCH Process (without shock)")

# Generate homoskedastic data (no volatility clustering)
print("\nGenerating homoskedastic data (no volatility clustering)...")
data_homo = generate_sample_data(n_samples=1000, homoskedastic=True)
    


Generating GARCH process data (with volatility clustering)...

Analyzing GARCH process without shock...

Analyzing GARCH Process (without shock)...

Estimated GARCH parameters:
omega: 0.0412
alpha: 0.0900
beta: 0.8584
persistence: 0.9484
log_likelihood: -1264.2782
aic: 2534.5565
bic: 2549.2797

Return series statistics:
mean: 0.0072
std: 0.8811
skewness: 0.0440
kurtosis: 0.3594
min: -3.3109
max: 3.1140
acf_squared_returns: 0.0362

Plotting returns and volatility for GARCH Process (without shock)...

Volatility Estimation Statistics:
Mean Absolute Error: 0.102950
Root Mean Square Error: 0.122959
Correlation between True and Estimated: 0.980134

Plotting evidence of volatility clustering for GARCH Process (without shock)...

Volatility Estimation Statistics:
Mean Absolute Error: 0.102950
Root Mean Square Error: 0.122959
Correlation between True and Estimated: 0.980134

Plotting evidence of volatility clustering for GARCH Process (without shock)...

Volatility Clustering Statistics:
Numb

![](plots/garch_process_without_shock_returns_volatility.png)

![](plots/garch_process_without_shock_qq_plot.png)

![](plots/garch_process_without_shock_returns_distribution.png)

![](plots/garch_process_without_shock_volatility_clustering.png)

## 2. Analyzing a Homoskedastic Process and comparing datasets...

In [16]:
analyzer_homo = analyze_data(data_homo, "Homoskedastic Process (no volatility clustering)")


Analyzing Homoskedastic Process (no volatility clustering)...

Estimated GARCH parameters:
omega: 0.0001
alpha: 0.0155
beta: 0.6422
persistence: 0.6577
log_likelihood: 2493.9651
aic: -4981.9302
bic: -4967.2069

Return series statistics:
mean: 0.0014
std: 0.0199
skewness: -0.0527
kurtosis: 0.0609
min: -0.0588
max: 0.0639
acf_squared_returns: 0.0033

Plotting returns and volatility for Homoskedastic Process (no volatility clustering)...

Volatility Estimation Statistics:
Mean Absolute Error: 0.121361
Root Mean Square Error: 0.121366
Correlation between True and Estimated: nan

Plotting evidence of volatility clustering for Homoskedastic Process (no volatility clustering)...

Volatility Estimation Statistics:
Mean Absolute Error: 0.121361
Root Mean Square Error: 0.121366
Correlation between True and Estimated: nan

Plotting evidence of volatility clustering for Homoskedastic Process (no volatility clustering)...

Volatility Clustering Statistics:
Number of significant lags: 3 out of 20
F

![](plots/homoskedastic_process_no_volatility_clustering_returns_volatility.png) 

![](plots/homoskedastic_process_no_volatility_clustering_qq_plot.png)

![](plots/homoskedastic_process_no_volatility_clustering_returns_distribution.png)

![](plots/homoskedastic_process_no_volatility_clustering_volatility_clustering.png)

## 3. Analyzing GARCH process with shock...

In [ ]:
shock_params = {
    'time': 500,      # Shock at day 500
    'magnitude': 8.0  # 8x normal volatility (increased from 5x)
}
data_garch_shock = generate_sample_data(
    n_samples=1000, 
    garch_params=garch_params,
    add_shock=True,
    shock_params=shock_params
)
analyzer_shock = analyze_data(data_garch_shock, "GARCH Process (with shock)")


Analyzing GARCH Process (with shock)...

Estimated GARCH parameters:
omega: 0.0294
alpha: 0.0916
beta: 0.8792
persistence: 0.9709
log_likelihood: -1332.0613
aic: 2670.1225
bic: 2684.8458

Return series statistics:
mean: 0.0190
std: 0.9759
skewness: 0.3590
kurtosis: 2.5267
min: -4.5812
max: 5.4026
acf_squared_returns: 0.0610

Plotting returns and volatility for GARCH Process (with shock)...

Volatility Estimation Statistics:
Mean Absolute Error: 0.120988
Root Mean Square Error: 0.165705
Correlation between True and Estimated: 0.971202

Plotting evidence of volatility clustering for GARCH Process (with shock)...

Volatility Estimation Statistics:
Mean Absolute Error: 0.120988
Root Mean Square Error: 0.165705
Correlation between True and Estimated: 0.971202

Plotting evidence of volatility clustering for GARCH Process (with shock)...

Volatility Clustering Statistics:
Number of significant lags: 15 out of 20
First lag autocorrelation: 0.0610
95% Confidence bound: ±0.0620
Weak or no evide

![](plots/garch_process_with_shock_returns_volatility.png)

![](plots/garch_process_with_shock_qq_plot.png)

![](plots/garch_process_with_shock_returns_distribution.png)

![](plots/garch_process_with_shock_volatility_clustering.png)

## 4. Analyzing shock impact...

In [18]:
shock_date = data_garch_shock.index[shock_params['time']]
visualizer = VolatilityVisualizer(data_garch_shock)
visualizer.plot_shock_analysis(shock_date)

# Compare persistence before and after shock
print("\nComparing GARCH persistence:")
print("\nPre-shock model:")
pre_shock_data = data_garch_shock.iloc[:shock_params['time']]
analyzer_pre = GARCHAnalyzer(pre_shock_data['returns'])
pre_params = analyzer_pre.fit_model()
print(f"Alpha: {pre_params['alpha']:.4f}")
print(f"Beta: {pre_params['beta']:.4f}")
print(f"Persistence: {pre_params['persistence']:.4f}")

print("\nPost-shock model:")
post_shock_data = data_garch_shock.iloc[shock_params['time']:]
analyzer_post = GARCHAnalyzer(post_shock_data['returns'])
post_params = analyzer_post.fit_model()
print(f"Alpha: {post_params['alpha']:.4f}")
print(f"Beta: {post_params['beta']:.4f}")
print(f"Persistence: {post_params['persistence']:.4f}")



Shock Impact Analysis:
Pre-shock statistics:
Mean volatility: 0.7796
Max volatility: 1.0889

Post-shock statistics:
Mean volatility: 1.5699
Max volatility: 2.3665
Peak-to-pre-shock ratio: 3.04

Shock half-life: 0 days

Comparing GARCH persistence:

Pre-shock model:
Alpha: 0.0712
Beta: 0.8456
Persistence: 0.9168

Post-shock model:
Alpha: 0.0946
Beta: 0.8784
Persistence: 0.9731


![](plots/_shock_analysis.png)

## 5. Analyzing real market data (S&P 500)...

In [19]:
market_data = get_market_data("^GSPC", years=4)
analyzer_market = analyze_data(market_data, "S&P 500")

# Compare simulated vs real data characteristics
print("\nComparison of Real vs Simulated Data:")
print("\nReal S&P 500 GARCH parameters:")
market_params = analyzer_market.fit_model()
print(f"Alpha: {market_params['alpha']:.4f}")
print(f"Beta: {market_params['beta']:.4f}")
print(f"Persistence: {market_params['persistence']:.4f}")

print("\nSimulated GARCH parameters:")
print(f"Alpha: {garch_params['alpha']:.4f}")
print(f"Beta: {garch_params['beta']:.4f}")
print(f"Persistence: {garch_params['alpha'] + garch_params['beta']:.4f}")

print("\nAnalysis complete! Key observations:")
print("1. GARCH process shows clear volatility clustering")
print("2. Homoskedastic process shows constant volatility")
print("3. GARCH process has more realistic fat-tailed returns distribution")
print("4. Shock creates a significant spike in volatility (8x normal)")
print("5. Post-shock volatility persistence may differ from pre-shock")
print("6. Autocorrelation in squared returns is much stronger in GARCH process")
print("7. Real market data shows similar volatility clustering patterns")
print("8. S&P 500 GARCH parameters indicate market volatility characteristics")


Market Data Summary:
Total observations: 1002
Date range: 2021-05-21 to 2025-05-16
Average daily return: 0.000422
Daily volatility: 0.011276

Analyzing S&P 500...

Estimated GARCH parameters:
omega: 0.0000
alpha: 0.0926
beta: 0.8920
persistence: 0.9846
log_likelihood: 3199.5227
aic: -6393.0454
bic: -6378.3161

Return series statistics:
mean: 0.0004
std: 0.0113
skewness: 0.1988
kurtosis: 6.9413
min: -0.0597
max: 0.0952
acf_squared_returns: 0.1592

Plotting returns and volatility for S&P 500...

Estimated Volatility Statistics:
Mean: 0.101135
Std Dev: 0.018013
Min: 0.072493
Max: 0.190174
Median: 0.096745

Plotting evidence of volatility clustering for S&P 500...

Volatility Clustering Statistics:
Number of significant lags: 7 out of 20
First lag autocorrelation: 0.1592
95% Confidence bound: ±0.0619
Strong evidence of volatility clustering (significant first-lag autocorrelation)

Plotting returns distribution for S&P 500...

Comparison of Real vs Simulated Data:

Real S&P 500 GARCH param

![](plots/s&p_500_returns_volatility.png)

![](plots/s&p_500_qq_plot.png)

![](plots/s&p_500_returns_distribution.png)

![](plots/s&p_500_volatility_clustering.png)